
Run this code on google colab to upsample **a video** using Fast-SRGAN pre-trained on on the DIV2k dataset. This generates a **x4 resolution video**

---


Download this notebook, upload it on google colab and then execute


---



Upload the input LR **mp4** video 


---


It is advisable to connect runtime to standard GPU


---


This is a SISR technique, hence it will upsample single image frame at a time. However, this method is much faster as compared to other SISR methods due to the addition of inverted residual networks in the standard SRGAN architecture

# Importing necessary libraries

In [ ]:
import os
import math
import cv2
import numpy as np
from tensorflow import keras

# Cloning the Fast-SRGAN github repository 

In [ ]:
!git clone https://github.com/HasnainRaz/Fast-SRGAN.git

Cloning into 'Fast-SRGAN'...
remote: Enumerating objects: 136, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 136 (delta 1), reused 0 (delta 0), pack-reused 130
Receiving objects: 100% (136/136), 95.31 MiB | 34.15 MiB/s, done.
Resolving deltas: 100% (65/65), done.


# Installing the required packages
You also have to uninstall the existing version of the h5py library and install the 2.10.0 version

In [ ]:
!pip install -r /content/Fast-SRGAN/requirements.txt

     |████████████████████████████████| 86.4 MB 48 kB/s 
     |████████████████████████████████| 28.2 MB 1.4 MB/s 
     |████████████████████████████████| 20.3 MB 42.9 MB/s 
     |████████████████████████████████| 50 kB 6.4 MB/s 
     |████████████████████████████████| 449 kB 44.7 MB/s 
     |████████████████████████████████| 3.8 MB 51.2 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=66f0eaad86ef555556820303777396cb1dcc87463b9b302fd5d5c3cbb15aec17
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.5
    Uninstalling numpy-1.21.5:
      Successfully uninstalled numpy-1.21.5
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninst

In [ ]:
!pip uninstall h5py
!pip install h5py==2.10.0

Found existing installation: h5py 3.1.0
Uninstalling h5py-3.1.0:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/h5py-3.1.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/h5py.libs/libaec-9c9e97eb.so.0.0.10
    /usr/local/lib/python3.7/dist-packages/h5py.libs/libhdf5-00e8fae8.so.200.0.0
    /usr/local/lib/python3.7/dist-packages/h5py.libs/libhdf5_hl-383c339f.so.200.0.0
    /usr/local/lib/python3.7/dist-packages/h5py.libs/libsz-e7aa62f5.so.2.0.1
    /usr/local/lib/python3.7/dist-packages/h5py.libs/libz-eb09ad1d.so.1.2.3
    /usr/local/lib/python3.7/dist-packages/h5py/*
Proceed (y/n)? y
  Successfully uninstalled h5py-3.1.0
     |████████████████████████████████| 2.9 MB 27.5 MB/s 


# Defining the necessary functions

In [ ]:
arr_names = []

In [ ]:
output_dir = '/content/outframes'
os.makedirs(output_dir)

In [ ]:
def video_to_frames(vid_path, kps):

  video = cv2.VideoCapture(vid_path)

  (major_ver, minor_ver, subminor_ver) = (cv2.__version__).split('.')

  if int(major_ver)  < 3 :
    fps = int(video.get(cv2.cv.CV_CAP_PROP_FPS))
  else :
    fps = int(video.get(cv2.CAP_PROP_FPS))

  if (fps == 0): 
    print('pls upload a different video')
    quit()

  print(fps)

  hop = round(fps/kps)
  if hop == 0:
    print('pls select such custom fps such that fps<=',fps)
    quit()

  video = cv2.VideoCapture(vid_path)

  frames_dir = '/content/framesdir'
  os.makedirs(frames_dir)

  currentframe = 0       

  while(True):
      ret,frame = video.read()
      if ret:
        if(currentframe%hop == 0):
          name = frames_dir + '/' + str(int(currentframe/hop)) + '.jpg'
          cv2.imwrite(name, frame)
          arr_names.append(output_dir + '/' + str(int(currentframe/hop)) + '.jpg') 
        currentframe += 1
      else:
        break
  
  video.release()

  return kps

In [ ]:
def frames_to_video(output_dir_path, kps):

  image = cv2.imread(output_dir_path + '/0.jpg')
  height, width, layers = image.shape
  size = (width,height)

  out = cv2.VideoWriter('/content/pretrained_FastSRGAN_output.mp4',cv2.VideoWriter_fourcc(*'DIVX'), kps , size)

  for i in range(len(arr_names)):
    out.write(cv2.imread(arr_names[i]))
  out.release()

# Converting the custom video to frames
Add the path(str) to your custom video and natural fps as arguments to the function- video_to_frames() given in the below cell

In [ ]:
fps = video_to_frames('/content/assault_part_2_shortened.mp4', 30)

30


# Apply the pre-trained generator on individual input frames


The output video frames will be saved in '/content/outframes' folder

Since we are cloning another github repo, we have to give the correct model file path to the 'load_model()' function in 'infer.py' 

To do this Follow the steps:-
1.   Click the files icon on the LHS bar of colab window
2.   Click the small arrow adjacent to the 'Fast-SRGAN' folder, and then double-click 'infer.py'
3.   In this notebook, replace the string 'models/generator.h5' with '/content/Fast-SRGAN/models/generator.h5'

In [ ]:
%%time
!python /content/Fast-SRGAN/infer.py --image_dir '/content/framesdir' --output_dir '/content/outframes'

CPU times: user 1.13 s, sys: 151 ms, total: 1.28 s
Wall time: 2min 25s


# Compiling the output frames to create the upsampled video

The output video will be saved as '/content/pretrained_FastSRGAN_output.mp4'

In [ ]:
frames_to_video(output_dir, fps)

# Removing the stored input and output frames
Run the cell below to remove the input and output frame directories

In [ ]:
for image in os.listdir('/content/framesdir'):
  os.remove('/content/framesdir' + '/' + image)
os.rmdir('/content/framesdir')

for image in os.listdir('/content/outframes'):
  os.remove('/content/outframes' + '/' + image)
os.rmdir('/content/outframes')